### Imports

In [1]:
import os
import pickle
import cv2

from doctr.io import DocumentFile
from doctr.models import ocr_predictor

from skimage import io

2022-11-25 15:42:47.034761: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-25 15:42:47.169343: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/venkat/BADRI/badrienv/lib/python3.8/site-packages/cv2/../../lib64:
2022-11-25 15:42:47.169364: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-25 15:42:47.190633: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when o

### DocTR model loading

In [2]:
input_dir  = './../data/images/'
output_dir = './../data/results/doctr/'

In [3]:
model = ocr_predictor(pretrained=True)

2022-11-25 15:42:52.832249: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/venkat/BADRI/badrienv/lib/python3.8/site-packages/cv2/../../lib64:
2022-11-25 15:42:52.832311: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/venkat/BADRI/badrienv/lib/python3.8/site-packages/cv2/../../lib64:
2022-11-25 15:42:52.832346: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/venkat/BADRI/badrienv/lib/python3.8/site-packages/cv2/../../lib64:
2022-11-25 15:42:52.832380: W tensorflow/stream

### Get Results

In [4]:
dir_list = os.listdir(input_dir)
results = {}
images = []

for image in dir_list:
    doc = DocumentFile.from_images(input_dir+image)
    result = model(doc)
    results[image] = result
    images.append(image)
    
# for i in range(len(results)):
#     results[i].show(docs[i])

In [5]:
dimensions = {}
for image,result in results.items():
    dim = tuple(reversed(result.pages[0].dimensions))
    values = []
    img = io.imread(os.path.join(input_dir,image))
    for block in result.pages[0].blocks:
        for line in block.lines:
            for word in line.words:
                geo = word.geometry
                a = list(a*b for a,b in zip(geo[0],dim))
                b = list(a*b for a,b in zip(geo[1],dim))
                values.append(a+b)
                cv2.rectangle(img, (int(a[0]), int(a[1])), (int(b[0]), int(b[1])), (0, 0, 255), 2)
    dimensions[image] = values
    io.imsave(output_dir + 'images/' + image, img)

### Store Results

In [6]:
with open(output_dir+'results.pkl', 'wb') as outp:  # Overwrites any existing file.
    pickle.dump(results, outp, pickle.HIGHEST_PROTOCOL)
    
with open(output_dir+'dimensions.pkl', 'wb') as outp:  # Overwrites any existing file.
    pickle.dump(dimensions, outp, pickle.HIGHEST_PROTOCOL)